In [82]:
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, data, seq_length, tokenizer):
#         self.data = data
#         self.seq_length = seq_length
#         self.tokenizer = tokenizer

# def load_data(file_path):
#     # Initialize an empty list to store caption pairs
#     caption_pairs = []

#     # Read lines from the file
#     with open(file_path, 'r', encoding='utf-8') as file:
#         lines = file.readlines()

#     # Iterate through lines to extract short and long captions
#     i = 0
#     while i < len(lines):
#         if lines[i].startswith('short caption:'):
#             short_caption = "prompt: "+lines[i].replace('short caption:', '').strip()
#             i += 1
#             if i < len(lines) and lines[i].startswith('long caption:'):
#                 long_caption = "response: "+lines[i].replace('long caption:', '').strip()
#                 caption_pairs.append((short_caption, long_caption))
#             else:
#                 # If the next line doesn't start with 'long caption:', skip the entry
#                 i += 1
#         else:
#             i += 1

#     return caption_pairs


#     def __len__(self):
#         if self.random_chunk:
#             return len(self.data) // self.seq_length - 1
#         else:
#             return (len(self.data) - self.offset) // self.seq_length

#     def __getitem__(self, index):
#         # Implement your logic to process each data point
#         # This might include tokenization, padding, and returning a dictionary with input and label

#         short_caption, long_caption = self.data[index]

#         encoding = self.tokenizer(
#             short_caption,
#             long_caption,
#             return_tensors="pt",
#             max_length=self.seq_length,
#             padding="max_length",
#             truncation=True,
#         )

#         return {
#             "input_ids": encoding["input_ids"].squeeze(),
#             "attention_mask": encoding["attention_mask"].squeeze(),
#         }


In [2]:
from transformers import (
    GPT2TokenizerFast,
    LlamaForCausalLM,
    LlamaConfig,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    LlamaTokenizer,
    DataCollatorForLanguageModeling,
)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import  Subset
from random import sample
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path
import wandb
from distillation_arguments import DistillationTrainingArguments

/opt/conda/envs/llama_finetune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
#############
LR = 2.5e-4
BATCH_SIZE = 16
SEQ_LENGTH = 128

TEMPERATURE = 2.0
ALPHA = 0.5
#############


PATH = Path("./")
tokenizer_path = PATH / "models/gpt-clean-32000.json"
tokenizer = GPT2TokenizerFast(tokenizer_file= str(tokenizer_path))
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = "<pad>"

# teacher_dir1 = PATH / 'models/Llama-360M'
# teacher_dir2 = PATH / 'models/gpt2'


MODEL_NAME = f'Baby-Llama2-144M'
MODEL_OUTPUT = Path('./models') /  MODEL_NAME
EVAL_SAMPLES = 8192


wandb_log = True



In [4]:
#tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = "<pad>"
teacher=None
teacher1 = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",ignore_mismatched_sizes=True,load_in_4bit=True,device_map="auto")
#teacher2 = GPT2LMHeadModel.from_pretrained(teacher_dir2,ignore_mismatched_sizes=True)
#teachers = [teacher1, teacher2]
teachers = [teacher1]


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.93s/it]


In [20]:
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [21]:
tokenizer

LlamaTokenizer(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [23]:
import pandas as pd 
from babylm_dataset import BabylmDataset
tokenizer.model_max_length = SEQ_LENGTH
print(tokenizer.model_max_length)
train_dataset = BabylmDataset("data/captions_clean", SEQ_LENGTH, tokenizer=tokenizer, random_chunk=True)
full_eval_dataset = BabylmDataset( "data/captions_dev_clean", SEQ_LENGTH, tokenizer=tokenizer, offset=0)
# eval_indices = sample(range(len(full_eval_dataset)), EVAL_SAMPLES)
# eval_dataset = Subset(full_eval_dataset, eval_indices)

eval_indices = sample(range(len(full_eval_dataset)), min(EVAL_SAMPLES, len(full_eval_dataset)))
eval_dataset = Subset(full_eval_dataset, eval_indices)


print(len(full_eval_dataset.data))


Keyword arguments {'add_special_tokens': False} not recognized.


128


Keyword arguments {'add_special_tokens': False} not recognized.


🔥 data/captions_clean/captions.jsonl len: 9421509
🔥 data/captions_clean/.ipynb_checkpoints/captions-checkpoint.jsonl len: 9421509
Saving data to data/captions_clean/tokenized_LlamaTokenizer_32000.pt


Keyword arguments {'add_special_tokens': False} not recognized.


🔥 data/captions_dev_clean/captions.jsonl len: 2350986
Saving data to data/captions_dev_clean/tokenized_LlamaTokenizer_32000.pt
2350986


In [24]:
print(len(full_eval_dataset.data))

2350986


In [32]:


config = LlamaConfig(
    vocab_size=32000,
    hidden_size=768,
    num_hidden_layers=20,
    intermediate_size=1024,
    num_attention_heads=8,
    bos_token_id=tokenizer.convert_tokens_to_ids("<s>"),
    eos_token_id=tokenizer.convert_tokens_to_ids("</s>"),
    pad_token_id=tokenizer.convert_tokens_to_ids("<pad>"),
    max_position_embeddings=2*SEQ_LENGTH,
)

student = LlamaForCausalLM(config)

In [33]:

text = "This is Llama"

encoded = tokenizer.encode(text)
decoded = tokenizer.decode(encoded)

print(f"Encoded IDs: {encoded}")
print(f"Decoded String: {decoded}")

# Access tokens using convert_ids_to_tokens
tokens = tokenizer.convert_ids_to_tokens(encoded)
print(f"Tokens: {tokens}")

Keyword arguments {'add_special_tokens': False} not recognized.


Encoded IDs: [1, 910, 338, 365, 29880, 3304]
Decoded String: <s> This is Llama
Tokens: ['<s>', '▁This', '▁is', '▁L', 'l', 'ama']


In [34]:
teachers = [teacher1]

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)


print(f'model num parameters: student  = {student.num_parameters()}')
print(f'model num parameters: teacher1 = {teacher1.num_parameters()}')



model num parameters: student  = 143555328
model num parameters: teacher1 = 6738415616


In [28]:
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_models=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teachers = teacher_models
        for teacher in self.teachers:
            # place each teacher on same device as student
            #self._move_model_to_device(teacher, self.model.device)
            teacher.eval()
        
        

    def compute_loss(self, model, inputs, return_outputs=False):
        # compute student output
        outputs_student = model(**inputs)
        student_loss = outputs_student.loss

        # compute teacher output
        with torch.no_grad():
            all_teacher_logits = []
            for teacher in self.teachers:
                outputs_teacher = teacher(**inputs)
                all_teacher_logits.append(outputs_teacher.logits)
            avg_teacher_logits = torch.stack(all_teacher_logits).mean(dim=0)

        # assert size
        # print("Student logits size:", outputs_student.logits.size())
        # print("Average teacher logits size:", avg_teacher_logits.size())

        #assert outputs_student.logits.size() == avg_teacher_logits.size()

        # Soften probabilities and compute distillation loss
        loss_function = nn.KLDivLoss(reduction="batchmean")
        loss_logits = (
            loss_function(
                F.log_softmax(outputs_student.logits / self.args.temperature, dim=-1),
                F.softmax(avg_teacher_logits / self.args.temperature, dim=-1),
            )
            * (self.args.temperature ** 2)
        )
        # Return weighted student loss
        loss = self.args.alpha * student_loss + (1.0 - self.args.alpha) * loss_logits
        return (loss, outputs_student) if return_outputs else loss



In [10]:

#if wandb_log:
wandb.login()
wandb.init(project='distilled_llama2', name=MODEL_NAME)



wandb: Currently logged in as: shahwaiz638. Use `wandb login --relogin` to force relogin


In [29]:
training_args = DistillationTrainingArguments(
    output_dir=MODEL_OUTPUT,
    overwrite_output_dir=True,
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    num_train_epochs=6,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=BATCH_SIZE,
    save_total_limit=1,  # Set to zero to avoid saving
    report_to="wandb",
    warmup_steps=200, 
    lr_scheduler_type="cosine",
    learning_rate=LR,
    logging_steps=20,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    weight_decay=0.1,
    alpha=ALPHA,
    temperature=TEMPERATURE,
    max_steps=1000,
)


trainer = DistillationTrainer(
        student,
        training_args,
        teacher_models=teachers,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,

    )


In [ ]:

trainer.train()


trainer.save_model(MODEL_OUTPUT)
tokenizer.save_pretrained(MODEL_OUTPUT)



/opt/conda/envs/llama_finetune/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [19]:
torch.cuda.empty_cache()